In [1]:
from ase.io import read, write
from mace.calculators.mace import MACECalculator


In [2]:
data = read('./forge/tests/resources/structures/gen_5_2024-11-28_test.xyz', index=':')

In [4]:
print(data[0].info)

{'structure_name': 'neb_30_structure_9001_step_0', 'REF_energy': -1107.27788642, 'REF_stress': array([ 0.00420268,  0.00039253, -0.00026311,  0.00039253,  0.00412004,
        0.00020602, -0.00026311,  0.00020602,  0.00508813])}


In [1]:
from forge.core.database import DatabaseManager
from ase.io import read
import os

# 1. Your AWS database credentials
db_config = {
    'database': {
        'dbname': 'test_database',  # The name of the database you created on RDS
        'user': 'myless',           # Replace with your RDS username
        'password': 'vcrtiwzr',     # Replace with your RDS password
        'host': 'database-vcrtiwzr.cfg4i4qmuc4m.us-east-1.rds.amazonaws.com',
        'port': 5432                # Default Postgres port unless you changed it
    }
}

# 2. Instantiate the DatabaseManager
db_manager = DatabaseManager(config_dict=db_config)

# (Optional) If you need to drop and recreate tables, uncomment:
# with db_manager.conn.cursor() as cur:
#     cur.execute("""
#         DROP TABLE IF EXISTS calculations CASCADE;
#         DROP TABLE IF EXISTS structures CASCADE;
#     """)
# db_manager.conn.commit()
# db_manager._initialize_tables()

In [2]:
data = read('./scratch/data/random_100_gen6.xyz', index=':')
#print(data[0])
#print(data[0].info)
#print(data[0].calc)

In [3]:
db_manager.add_structures_from_xyz(
    xyz_file='./scratch/data/random_100_gen6.xyz',
    skip_duplicates=True,
    default_model_type="vasp-static"
)

[INFO] Found 100 frames in ./scratch/data/random_100_gen6.xyz

[INFO] Processing frame 0 in ./scratch/data/random_100_gen6.xyz...
Executing query: SELECT structure_id FROM structures WHERE 1=1 AND composition ?| %s
With params: ['{Cr,Zr,Ti,W,V}']
Found 0 matching structures
[INFO] Added structure, ID=1
[INFO] Added calculation, ID=1

[INFO] Processing frame 1 in ./scratch/data/random_100_gen6.xyz...
Executing query: SELECT structure_id FROM structures WHERE 1=1 AND composition ?| %s
With params: ['{Cr,Zr,Ti,W,V}']
Found 1 matching structures
[INFO] Added structure, ID=2
[INFO] Added calculation, ID=2

[INFO] Processing frame 2 in ./scratch/data/random_100_gen6.xyz...
Executing query: SELECT structure_id FROM structures WHERE 1=1 AND composition ?| %s
With params: ['{Cr,Zr,Ti,W,V}']
Found 2 matching structures
[INFO] Added structure, ID=3
[INFO] Added calculation, ID=3

[INFO] Processing frame 3 in ./scratch/data/random_100_gen6.xyz...
Executing query: SELECT structure_id FROM structure

In [ ]:
# 1. Clear/Drop Tables
# -------------------
# BE CAREFUL: This will delete all data!
with db_manager.conn.cursor() as cur:
    cur.execute("""
        DROP TABLE IF EXISTS calculations CASCADE;
        DROP TABLE IF EXISTS structures CASCADE;
    """)
db_manager.conn.commit()
db_manager._initialize_tables()

In [ ]:

# 2. View Tables
# -------------
# Count total structures and calculations
with db_manager.conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM structures")
    struct_count = cur.fetchone()[0]
    cur.execute("SELECT COUNT(*) FROM calculations")
    calc_count = cur.fetchone()[0]
print(f"Database contains {struct_count} structures and {calc_count} calculations")

# View recent structures
with db_manager.conn.cursor() as cur:
    cur.execute("""
        SELECT structure_id, formula, metadata->>'generation' as gen 
        FROM structures 
        ORDER BY structure_id DESC 
        LIMIT 5
    """)
    print("\nRecent structures:")
    for row in cur.fetchall():
        print(f"ID: {row[0]}, Formula: {row[1]}, Generation: {row[2]}")

# View calculations for a specific structure
def view_calculations(db_manager, structure_id):
    with db_manager.conn.cursor() as cur:
        cur.execute("""
            SELECT calculation_id, model_type, energy, status
            FROM calculations
            WHERE structure_id = %s
        """, (structure_id,))
        print(f"\nCalculations for structure {structure_id}:")
        for row in cur.fetchall():
            print(f"Calc ID: {row[0]}, Model: {row[1]}, Energy: {row[2]}, Status: {row[3]}")


In [ ]:
# 3. Search Examples
# ----------------
# Find structures with specific composition (using existing method)
cr_structures = db_manager.find_structures(
    elements=['Cr'],  # Must contain Cr
    composition_constraints={
        'Cr': (0.05, 1.0)  # At least 5% Cr
    }
)
print(f"\nFound {len(cr_structures)} structures with >5% Cr")

# Find structures with high forces
high_force_structures = db_manager.find_structures(
    calculation_constraints={
        'model_type': 'vasp-static',
        'forces_max_magnitude': 0.5  # eV/Å
    }
)
print(f"\nFound {len(high_force_structures)} structures with forces >0.5 eV/Å")

# Find structures from a specific generation
gen5_structures = db_manager.find_structures(
    metadata_constraints={
        'generation': 5
    }
)
print(f"\nFound {len(gen5_structures)} structures from generation 5")

# Combine multiple constraints
specific_structures = db_manager.find_structures(
    elements=['V', 'Cr'],
    composition_constraints={
        'V': (0.4, 0.6),    # 40-60% V
        'Cr': (0.1, 0.2)    # 10-20% Cr
    },
    metadata_constraints={
        'generation': 5,
        'structure_type': 'vac'
    },
    calculation_constraints={
        'model_type': 'vasp-static',
        'energy': (-1200, -1000)  # Energy range in eV
    }
)
print(f"\nFound {len(specific_structures)} structures matching all criteria")


In [ ]:

# 4. Examine Structure Details
# --------------------------
# Get full details for a specific structure
def examine_structure(db_manager, structure_id):
    with db_manager.conn.cursor() as cur:
        # Get structure details
        cur.execute("""
            SELECT formula, composition, metadata
            FROM structures
            WHERE structure_id = %s
        """, (structure_id,))
        row = cur.fetchone()
        if row:
            print(f"\nStructure {structure_id}:")
            print(f"Formula: {row[0]}")
            print(f"Composition: {row[1]}")
            print(f"Metadata: {row[2]}")
            
            # Get associated calculations
            cur.execute("""
                SELECT model_type, energy, status, metadata
                FROM calculations
                WHERE structure_id = %s
            """, (structure_id,))
            print("\nCalculations:")
            for calc in cur.fetchall():
                print(f"- {calc[0]}: E={calc[1]:.3f} eV, Status: {calc[2]}")
        else:
            print(f"No structure found with ID {structure_id}")

# Usage example:
if specific_structures:
    examine_structure(db_manager, specific_structures[0])